In [3]:
!pip install setfit

In [ ]:
# SETFIT ERROR as of December 2024
# AttributeError: 'CallbackHandler' object has no attribute 'tokenizer'
# https://github.com/huggingface/setfit/issues/564
# solution is to install older version of transformers
!pip install transformers==4.45.2

In [4]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [5]:
from datasets import load_dataset

dataset = load_dataset("data-is-better-together/10k_prompts_ranked")

dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 10331
    })
})

In [6]:
dataset_tt = dataset["train"].train_test_split(test_size=0.5)

In [7]:
dataset_tt

DatasetDict({
    train: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 5165
    })
    test: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 5166
    })
})

In [8]:
from setfit import sample_dataset

# Mimic only having a few labeled examples from each class

- here i'm taking 20 of each class (`topic` is the `label` in our current dataset, will rename later)
- so `sample_dataset` from setfit should take 20 **of each possible label** so we should see 20 * 10 = 200 samples in train dataset below (since there are 10 labels in this dataset)

## Update

doing 20 samples for all 10 classes, with 500 steps, i get accuracy of 0.505 after 20 mins

- the advice is that you get better results with more data, so i'll try 100 samples of each now

In [9]:
#train_dataset = sample_dataset(dataset_tt["train"], label_column="topic", num_samples=20) # get 0.505 accuracy with 500 steps

train_dataset = sample_dataset(dataset_tt["train"], label_column="topic", num_samples=100) 

In [10]:
train_dataset

Dataset({
    features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
    num_rows: 929
})

In [11]:
test_dataset = dataset_tt["test"]

In [12]:
test_dataset

Dataset({
    features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
    num_rows: 5166
})

In [13]:
from setfit import TrainingArguments

args = TrainingArguments(
    batch_size=32,
    #num_epochs=1, # adjusted down from 10
    max_steps=500, # get 40% accuracy at 50 steps on the test dataset, tried small number to make sure it works
    report_to="none",
)

"""
The num_epochs and max_steps arguments are frequently used to increase and decrease the number of total training steps. Consider that with SetFit, better performance is reached with more data, not more training! Don’t be afraid to train for less than 1 epoch if you have a lot of data.
"""

'\nThe num_epochs and max_steps arguments are frequently used to increase and decrease the number of total training steps. Consider that with SetFit, better performance is reached with more data, not more training! Don’t be afraid to train for less than 1 epoch if you have a lot of data.\n'

In [14]:
from setfit import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    column_mapping={"prompt":"text", "topic":"label"} # <--- SETFIT EXPECTS text AND label RESPECTIVELY
)

Applying column mapping to the training dataset


Map:   0%|          | 0/929 [00:00<?, ? examples/s]

In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [16]:
import wandb
wandb.init(mode="disabled")

/opt/conda/lib/python3.10/site-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)


In [17]:
trainer.train()

***** Running training *****
  Num unique pairs = 16000
  Batch size = 32
  Num epochs = 1
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Step,Training Loss
1,0.234300
50,0.237100
100,0.221400
150,0.205200
200,0.184400
250,0.171100
300,0.147000
350,0.137200
400,0.124300
450,0.123100


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [18]:
trainer.evaluate(test_dataset)

Applying column mapping to the evaluation dataset
***** Running evaluation *****


Batches:   0%|          | 0/162 [00:00<?, ?it/s]

{'accuracy': 0.5642663569492837}

In [19]:
model.predict(["write a new novel for me", "what is the best place to go to visit in italy?"])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array(['Literature and Arts', 'Travel and Leisure'], dtype='<U22')

# Summary

- with 20 samples and 500 training steps, get 50.5 % accuracy (not shown in notebook, deleted test run)
- with 100 samples from each class, and 500 training steps, get 56.4 % accuracy

## Try much larger number of samples, then will compare with finetuning a BERT type model

- use 200 samples from each class

In [20]:
train_dataset2 = sample_dataset(dataset_tt["train"], label_column="topic", num_samples=200) 

In [21]:
args2 = TrainingArguments(
    batch_size=32,
    #num_epochs=1, # adjusted down from 10
    max_steps=500, # get 40% accuracy at 50 steps on the test dataset, tried small number to make sure it works
    report_to="none",
)

trainer2 = Trainer(
    model=model,
    args=args2,
    train_dataset=train_dataset2,
    column_mapping={"prompt":"text", "topic":"label"} # <--- SETFIT EXPECTS text AND label RESPECTIVELY
)

Applying column mapping to the training dataset
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map:   0%|          | 0/1797 [00:00<?, ? examples/s]

In [22]:
trainer2.train()

***** Running training *****
  Num unique pairs = 16000
  Batch size = 32
  Num epochs = 1
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Step,Training Loss
1,0.183000
50,0.173900
100,0.159200
150,0.149700
200,0.139500
250,0.125600
300,0.111000
350,0.107300
400,0.095700
450,0.095200


Batches:   0%|          | 0/57 [00:00<?, ?it/s]

In [23]:
trainer2.evaluate(test_dataset)

Applying column mapping to the evaluation dataset
***** Running evaluation *****


Batches:   0%|          | 0/162 [00:00<?, ?it/s]

{'accuracy': 0.5687185443283004}

## Comments

- 56.8 with way more samples compared to 56.4 with 100 samples per label

# Do a trained distilBERT comparison

In [87]:
dataset_tt

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5165
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5166
    })
})

In [62]:
dataset_tt["train"].features

{'text': Value(dtype='string', id='field'),
 'quality': [{'user_id': Value(dtype='string', id='question'),
   'value': Value(dtype='string', id='suggestion'),
   'status': Value(dtype='string', id='question')}],
 'metadata': Value(dtype='string', id='metadata'),
 'avg_rating': Value(dtype='float64', id=None),
 'num_responses': Value(dtype='int64', id=None),
 'agreement_ratio': Value(dtype='float64', id=None),
 'raw_responses': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'kind': Value(dtype='string', id=None),
 'cluster_description': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None)}

In [28]:
!pip install evaluate accelerate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [120]:
MODEL_NAME = "distilbert/distilbert-base-uncased"

In [119]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [90]:
dataset_tt

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5165
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5166
    })
})

In [66]:
dataset_tt = dataset_tt.rename_column("prompt","text")

ValueError: Original column name prompt not in the dataset. Current columns in the dataset: ['text', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'label']

In [67]:
dataset_tt = dataset_tt.rename_column("topic","label")

ValueError: Original column name topic not in the dataset. Current columns in the dataset: ['text', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'label']

In [68]:
dataset_tt

DatasetDict({
    train: Dataset({
        features: ['text', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'label'],
        num_rows: 5165
    })
    test: Dataset({
        features: ['text', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'label'],
        num_rows: 5166
    })
})

In [69]:
dataset_tt = dataset_tt.remove_columns(['quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description'])

dataset_tt

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 5165
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5166
    })
})

In [108]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length") # adding padding = "max_length" here

In [109]:
tokenized_ds = dataset_tt.map(preprocess_function, batched=True)

Map:   0%|          | 0/5165 [00:00<?, ? examples/s]

Map:   0%|          | 0/5166 [00:00<?, ? examples/s]

# deleted data collator, causing bugs i can't understand why (it's literally the docs on HF site)

In [110]:
import evaluate

accuracy = evaluate.load("accuracy")

In [111]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# Get labels, they don't seem to be accessible in dataset itseflf

In [112]:
TVT_KEYS = ("train","test")
ALL_LABELS_NAMES = set()

for key in TVT_KEYS:
    for sample in dataset_tt[key]:
        ALL_LABELS_NAMES.add(sample["label"])

In [113]:
ALL_LABELS_NAMES = sorted(ALL_LABELS_NAMES)

In [114]:
print(ALL_LABELS_NAMES, len(ALL_LABELS_NAMES))

['Business and Marketing', 'Environmental Issues', 'Health and Wellness', 'Legal and Government', 'Literature and Arts', 'Math', 'Others', 'Science and Technology', 'Software Development', 'Travel and Leisure'] 10


In [115]:
label2id = {label_name:idx for idx,label_name in enumerate(ALL_LABELS_NAMES)}

print(label2id)

{'Business and Marketing': 0, 'Environmental Issues': 1, 'Health and Wellness': 2, 'Legal and Government': 3, 'Literature and Arts': 4, 'Math': 5, 'Others': 6, 'Science and Technology': 7, 'Software Development': 8, 'Travel and Leisure': 9}


In [116]:
id2label = {idx:label_name for label_name,idx in label2id.items()}

print(id2label)

{0: 'Business and Marketing', 1: 'Environmental Issues', 2: 'Health and Wellness', 3: 'Legal and Government', 4: 'Literature and Arts', 5: 'Math', 6: 'Others', 7: 'Science and Technology', 8: 'Software Development', 9: 'Travel and Leisure'}


In [117]:
NUM_LABELS = len(label2id)

print(NUM_LABELS)

10


In [121]:
from transformers import DistilBertForSequenceClassification, TrainingArguments, Trainer

model = DistilBertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=NUM_LABELS, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [122]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5165
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5166
    })
})

In [58]:
#tokenized_ds = tokenized_ds.remove_columns(tokenized_ds["train"].column_names)

In [83]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5165
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5166
    })
})

In [124]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding="max_length", truncation=True)

In [125]:
tokenized_ds["train"]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 5165
})

In [128]:
tokenized_ds["train"][0]

{'text': 'there are only two days a year when there will never be a professional major league game (NFL, MLB, NHL, NBA).  what are these days?',
 'label': 'Literature and Arts',
 'input_ids': [101,
  2045,
  2024,
  2069,
  2048,
  2420,
  1037,
  2095,
  2043,
  2045,
  2097,
  2196,
  2022,
  1037,
  2658,
  2350,
  2223,
  2208,
  1006,
  5088,
  1010,
  10901,
  1010,
  7097,
  1010,
  6452,
  1007,
  1012,
  2054,
  2024,
  2122,
  2420,
  1029,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [127]:
training_args = TrainingArguments(
    output_dir="finetuned-prompt-classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    #data_collator=data_collator, # <--- deleted during debug, doesn't work and answers are unclear on HF github
    compute_metrics=compute_metrics,
)

training_metrics = trainer.train()

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).